In [19]:
import pandas as pd

In [20]:
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'

In [21]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [22]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [23]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,1,2024-05-16,4.127295,221.918352,NaN
1,1,2024-05-29,NaN,NaN,1906.18
2,1,2024-05-31,45.972261,2338.489409,970.59
3,1,2024-06-05,63.240335,2836.576690,2483.70
4,1,2024-06-10,48.153913,2486.759690,2421.99
...,...,...,...,...,...
4584,583,2024-08-15,NaN,NaN,156.68
4585,583,2024-08-19,NaN,NaN,326.84
4586,583,2024-08-29,NaN,NaN,439.29
4587,583,2024-09-04,NaN,NaN,582.33


In [24]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador
        
    #print(df_param)
    #print(suma_acumulador)
    df_param['ENTREGAS'] = suma_acumulador
    
    print(df_param)
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [25]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [26]:
list_df = []

In [27]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

    COD_COS      FECHA        AREA     ESTIMADO  ENTREGADO  ENTREGAS
0         1 2024-05-16    4.127295   221.918352       0.00      0.00
1         1 2024-05-29    0.000000     0.000000    1906.18       NaN
2         1 2024-05-31   45.972261  2338.489409     970.59   2876.77
3         1 2024-06-05   63.240335  2836.576690    2483.70   2483.70
4         1 2024-06-10   48.153913  2486.759690    2421.99   2421.99
5         1 2024-06-18   20.112090   815.611812    2230.23   2230.23
6         1 2024-06-20   29.470226  2021.733284       0.00      0.00
7         1 2024-07-20   10.552630   619.266573    1136.33   1136.33
8         1 2024-07-25   33.115243  1651.084930    1916.31   1916.31
9         1 2024-08-11  111.015957  7411.451546    6811.41   6811.41
10        1 2024-08-15   21.668291  1388.784784    1112.92   1112.92
11        1 2024-08-19   22.008380  1162.986991     910.26    910.26
12        1 2024-08-29   84.790815  3131.051326    3644.64   3644.64
13        1 2024-09-04   44.708430

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

     COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
557       69 2024-07-17  8.423921  402.185065     290.32    290.32
558       69 2024-07-20  0.000000    0.000000     200.03       NaN
559       69 2024-07-25  9.930336  447.790469     166.89    366.92
560       69 2024-08-11  9.984130  612.003850     579.97    579.97
561       69 2024-08-15  0.000000    0.000000      21.47     21.47
     COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
562       70 2024-05-29  0.000000    0.000000     465.87       NaN
563       70 2024-05-31  6.672224  308.731791     119.95    585.82
564       70 2024-06-05  2.369076  152.748410     159.80    159.80
565       70 2024-06-10  1.980026  112.772445     312.62    312.62
566       70 2024-06-18  4.099693  213.413525     317.21    317.21
567       70 2024-06-20  0.000000    0.000000      22.38       NaN
568       70 2024-07-17  4.209322  194.979580     354.31    376.69
569       70 2024-07-20  1.694628   94.074669      51.36     5

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

...

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

      COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
3747      425 2024-07-17  0.000000    0.000000     306.62       NaN
3748      425 2024-08-11  5.852579   90.776911     433.99    740.61
3749      425 2024-08-15  0.897276   56.652289     176.71    176.71
3750      425 2024-08-19  0.000000    0.000000     127.71       NaN
3751      425 2024-08-29  9.601468  468.378771     724.02    851.73
3752      425 2024-09-04  0.000000    0.000000      74.21     74.21
      COD_COS      FECHA      AREA   ESTIMADO  ENTREGADO  ENTREGAS
3753      426 2024-07-25  0.000000   0.000000      43.97       NaN
3754      426 2024-08-11  0.822657  50.187252       0.00     43.97
      COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
3755      427 2024-07-17  4.606361  137.029129      85.82     85.82
3756      427 2024-07-20  3.041744  136.873096       0.00      0.00
3757      427 2024-07-25  0.000000    0.000000      44.11       NaN
3758      427 2024-08-11  3.023528  136.053437     

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

      COD_COS      FECHA       AREA    ESTIMADO  ENTREGADO  ENTREGAS
4288      536 2024-05-31  19.120843  860.526862       0.00      0.00
4289      536 2024-08-29  20.082668    0.000000     242.57    242.57
      COD_COS      FECHA      AREA    ESTIMADO  ENTREGADO  ENTREGAS
4290      537 2024-05-29  0.000000    0.000000     126.67       NaN
4291      537 2024-05-31  1.651028  126.715989      60.73    187.40
4292      537 2024-06-05  1.526164  106.194293     103.50    103.50
4293      537 2024-06-10  2.259985  145.996418     119.64    119.64
4294      537 2024-06-18  1.880973  161.223230     203.35    203.35
4295      537 2024-06-20  0.000000    0.000000      44.23       NaN
4296      537 2024-07-17  7.537007  449.751460     527.76    571.99
4297      537 2024-07-20  0.000000    0.000000      32.90       NaN
4298      537 2024-07-25  2.508363   91.311331     126.65    159.55
4299      537 2024-08-11  0.000000    0.000000     491.92       NaN
4300      537 2024-08-15  0.000000    0.00000

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18752\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [28]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [29]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,1,2024-05-16,4.127295,221.918352,0.00
1,1,2024-05-31,45.972261,2338.489409,2876.77
2,1,2024-06-05,63.240335,2836.576690,2483.70
3,1,2024-06-10,48.153913,2486.759690,2421.99
4,1,2024-06-18,20.112090,815.611812,2230.23
...,...,...,...,...,...
3249,581,2024-08-19,4.186267,0.000000,88.91
3250,581,2024-08-29,1.243801,0.000000,72.09
3251,582,2024-09-04,0.000000,0.000000,42.11
3252,583,2024-09-04,0.000000,0.000000,1505.14


In [30]:
df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')